In [2]:
import h5py
import numpy as np
import pandas as pd
import json
import yaml
from ludwig import LudwigModel
import copy
import ray
from ludwig.utils.misc import merge_dict
import re

In [3]:
pattern = "^{{.*}}$"
list_index_pattern = "^\[\d+\]$"


def get_keys_from_list(l, path=""):
    parameters = []

    for index, v in enumerate(l):
        if isinstance(l[index], str):
             if re.match(pattern, l[index], flags=0) is not None:
                parameters.append([path + "[" + index + "]", v])

        elif isinstance(l[index], dict):
            p = get_keys_from_dict(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)
        elif isinstance(l[index], list):
            p = get_keys_from_list(l[index], path+ "[" + str(index) + "]" +"->" )
            if p:
                for l in p:
                    parameters.append(l)

    return parameters


def get_keys_from_dict(dct, path=""):
    parameters = []
    for k, v in dct.items():
        if isinstance(dct[k], str):
            if re.match(pattern, dct[k], flags=0) is not None:
                parameters.append([path + k,dct[k]])

        elif isinstance(dct[k], dict):
            p = get_keys_from_dict(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)

        elif isinstance(dct[k], list):
            p = get_keys_from_list(dct[k], path+k+"->" )
            if p:
                for l in p:
                    parameters.append(l)

    return parameters



def update_param(dct, path, value):
    if len(path) == 1:
        dct[path[0]] = value
    else:
        l = re.match(list_index_pattern, path[0], flags=0)
        if l is not None and isinstance(dct, list):
            index = int(l.group(0)[1:-1])
            update_param(dct[index], path[1:], value)
        else:
             update_param(dct[path[0]], path[1:], value)


def build_model(base_model, config):
    for k, v in config.items():
        p = k.split('->')
        update_param(base_model, p, v)
    return base_model

In [8]:
from ray.tune import grid_search

def build_search_space(annotated_model):
    pattern = "^{{(.*)}}$"
    grid_search_space = {}
    keys = get_keys_from_dict(annotated_model)
    for a in keys:
        grid_search_space[a[0]]= grid_search(yaml.load(re.match(pattern, a[1], flags=0)[1]))

    return grid_search_space


In [11]:
data = '/Users/bm255022/Projects/Teradata/ludwig/examples/hyperparameters/titanic.hdf5'
metadata = '/Users/bm255022/Projects/Teradata/ludwig/examples/hyperparameters/titanic.json'

def train(base, config, reporter):
    new_model_def = build_model(base, config)
    model = LudwigModel(new_model_def)
    train_stats = model.train(data_hdf5=data, train_set_metadata_json=metadata)
    return reporter(mean_accuracy=train_stats['validation']['Survived']['accuracy'][-1], done=True)


In [12]:
from ray.tune import register_trainable, grid_search, run_experiments

ray.shutdown()
ray.init()

with open("titanic.yaml", 'r') as stream:
    annotated_model = yaml.load(stream)

grid_search_space = build_search_space(annotated_model)
register_trainable('train', lambda cfg, rptr: train(annotated_model, cfg, rptr))
run_experiments({'my_experiment': {
    'run': 'train',
    'stop': {'mean_accuracy': 0.9},
    'config': grid_search_space}
    })


2019-03-25 11:49:49,580	WARNING worker.py:1381 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-03-25 11:49:49,583	INFO node.py:439 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-25_11-49-49_63973/logs.
2019-03-25 11:49:49,695	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:32397 to respond...
2019-03-25 11:49:49,817	INFO services.py:364 -- Waiting for redis server at 127.0.0.1:37414 to respond...
2019-03-25 11:49:49,821	INFO services.py:761 -- Starting Redis shard with 3.44 GB max memory.
2019-03-25 11:49:49,852	INFO services.py:1449 -- Starting the Plasma object store with 5.15 GB memory using /tmp.
/Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please re

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/8 CPUs, 0/0 GPUs
Unknown memory usage. Please run `pip install psutil` (or ray[debug]) to resolve)
Result logdir: /Users/bm255022/ray_results/my_experiment
Number of trials: 24 ({'RUNNING': 1, 'PENDING': 23})
PENDING trials:
 - train_1_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8:	PENDING
 - train_2_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=8:	PENDING
 - train_3_combiner->fc_size=24,input_features->[6]->representation=dense,training->batch_size=8:	PENDING
 - train_4_combiner->fc_size=48,input_features->[6]->representation=dense,training->batch_size=8:	PENDING
 - train_5_combiner->fc_size=64,input_features->[6]->representation=dense,training->batch_size=8:	PENDING
 - tr

(pid=65020) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
(pid=65020)   "Use dataset[()] instead.", H5pyDeprecationWarning)
(pid=65020) 2019-03-25 11:50:21.834228: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Result for train_2_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=8:
  date: 2019-03-25_11-50-22
  done: true
  experiment_id: c7dde8efe039470cb272f1fb04170fbc
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.8148148148148148
  node_ip: 10.0.9.192
  pid: 65027
  time_since_restore: 9.038642883300781
  time_this_iter_s: 9.038642883300781
  time_total_s: 9.038642883300781
  timestamp: 1553529022
  timesteps_since_restore: 0
  training_iteration: 1
  
== Status ==
Using FIFO scheduling

(pid=65023) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
Result for train_6_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=16:
  date: 2019-03-25_11-50-28
  done: true
  experiment_id: 922e956f7f9b4d1faae8eb0d27ab83ce
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.7654320987654321
  node_ip: 10.0.9.192
  pid: 65025
  time_since_restore: 14.070816040039062
  time_this_iter_s: 14.070816040039062
  time_total_s: 14.070816040039062
  timestamp: 1553529028
  timesteps_since_restore: 0
  training_iteration: 1
  
(pid=65033) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
(pid=65033)   return f(*args, **kwds)
(pid=65025) WARNING: No

(pid=65032) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
(pid=65032)   "Use dataset[()] instead.", H5pyDeprecationWarning)
(pid=65032) 2019-03-25 11:50:51.887677: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=65029) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
(pid=65029)   "Use dataset[()] instead.", H5pyDeprecationWarning)
Result for train_8_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=16:
  date: 2019-03-25_11-50-53
  done: true
  experiment_id: 1281afc9e6bc49f3baa39862a66212f4
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.7901234567901234

...

(pid=65033) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=65176) /Users/bm255022/Projects/Teradata/ludwig/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
(pid=65176)   return f(*args, **kwds)
Result for train_15_combiner->fc_size=24,input_features->[6]->representation=dense,training->batch_size=32:
  date: 2019-03-25_11-51-16
  done: true
  experiment_id: 8a2f66c80740465589e27ae3408b1fc1
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.8024691358024691
  node_ip: 10.0.9.192
  pid: 65031
  time_since_restore: 17.19588828086853
  time_this_iter_s: 17.19588828086853
  time_total_s: 17.19588828086853
  timestamp: 1553529076
  timesteps_since_restore: 0
  training_iteration: 1
  
== Status ==
Using FIFO sc

(pid=65157) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
Result for train_17_combiner->fc_size=64,input_features->[6]->representation=dense,training->batch_size=32:
  date: 2019-03-25_11-51-34
  done: true
  experiment_id: c4fa68930bf94579808bbff8a5516195
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.7901234567901234
  node_ip: 10.0.9.192
  pid: 65158
  time_since_restore: 11.134928226470947
  time_this_iter_s: 11.134928226470947
  time_total_s: 11.134928226470947
  timestamp: 1553529094
  timesteps_since_restore: 0
  training_iteration: 1
  
(pid=65158) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
Result for train_20_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=64:
  

Result for train_18_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=64:
  date: 2019-03-25_11-51-46
  done: true
  experiment_id: 2276f9dea4894dcfab92f3c3828fcf48
  hostname: MUSBM255022-163
  iterations_since_restore: 1
  mean_accuracy: 0.7777777777777778
  node_ip: 10.0.9.192
  pid: 65160
  time_since_restore: 22.138476133346558
  time_this_iter_s: 22.138476133346558
  time_total_s: 22.138476133346558
  timestamp: 1553529106
  timesteps_since_restore: 0
  training_iteration: 1
  
(pid=65168) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
(pid=65160) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
Result for train_23_combiner->fc_size=64,input_features->[6]->representation=dense,training->batch_size=64:
  

[train_0_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=8,
 train_1_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=8,
 train_2_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=8,
 train_3_combiner->fc_size=24,input_features->[6]->representation=dense,training->batch_size=8,
 train_4_combiner->fc_size=48,input_features->[6]->representation=dense,training->batch_size=8,
 train_5_combiner->fc_size=64,input_features->[6]->representation=dense,training->batch_size=8,
 train_6_combiner->fc_size=24,input_features->[6]->representation=sparse,training->batch_size=16,
 train_7_combiner->fc_size=48,input_features->[6]->representation=sparse,training->batch_size=16,
 train_8_combiner->fc_size=64,input_features->[6]->representation=sparse,training->batch_size=16,
 train_9_combiner->fc_size=24,input_features->[6]->representation=dense,training->batch_size=16,
 train_10_combiner->fc_size=48

(pid=65174) WARNING: Not monitoring node memory since `psutil` is not installed. Install this with `pip install psutil` (or ray[debug]) to enable debugging of memory-related crashes.
